In [ ]:
!pip install -q efficientnet

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split

## ______________ БЛОК С ИМПОРТАМИ АРХИТЕКТУР ____________________
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201 
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
from tensorflow.keras.applications.nasnet import NASNetLarge
from efficientnet.tfkeras import EfficientNetB7, EfficientNetL2
## ______________ КОНЕЦ БЛОКА С ИМПОРТАМИ АРХИТЕКТУР ____________________

# импорт других полезных инструментов: слоев, оптимизаторов, функций обратной связи
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.layers import Dropout, BatchNormalization, GaussianDropout
from efficientnet.tfkeras import EfficientNetB5
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications.xception import Xception
#  библиотека для работы с наборами данных на Kaggle
from kaggle_datasets import KaggleDatasets
import re
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
print("Tensorflow version " + tf.__version__)

# Определяем, какой ускоритель можем использовать

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Проверяем существующее оборудование и выбираем соответствующую стратегию использования вычислений
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # онаружение TPU. Параметры не требуются если установленна переменная окружения TPU_NAME. На Kaggle это всегда True.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # если TPU отсутствует, то испльзуем стратегию по умолчанию для TF (CPU or GPU)

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Путь к данным. Если работаете на Google Colaboratory, то замените KaggleDatasets().get_gcs_path() на путь к данным, который будет у вас
GCS_DS_PATH = KaggleDatasets().get_gcs_path("plant-pathology-2020-fgvc7")

# Конфигурация
EPOCHS = 40
BATCH_SIZE = 8 * strategy.num_replicas_in_sync

# Загружаем метки классов и пути к изображениям

In [ ]:
# функция, которая превращает айди картинки в полный путь к ней
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values

train_labels = train.loc[:, 'healthy':].values

## если планируете обучать модель с валидирующим набором данных
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
     train_paths, train_labels, test_size=0.15, random_state=2020)

## Создаем объекты наборов данных

In [ ]:
# устанавливаем глобальные переменные
img_size = 768

# функция, которая читает изображение из файла и преобразовывает его к нужному размеру, а так же нормализует
def decode_image(filename, label=None, image_size=(img_size, img_size)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label
# функция расширения данных
def data_augment(image, label=None, seed=2020):
    image = tf.image.random_flip_left_right(image, seed=seed)
    image = tf.image.random_flip_up_down(image, seed=seed)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
## если планируете обучать модель с валидирующим набором данных
valid_dataset = (
     tf.data.Dataset
     .from_tensor_slices((valid_paths, valid_labels))
     .map(decode_image, num_parallel_calls=AUTO)
     .batch(BATCH_SIZE)
     .cache()
     .prefetch(AUTO)
 )

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

# Построить модель на TPU (или GPU, или CPU...) с Tensorflow 2.1!

In [ ]:
def get_model(use_model):
    base_model =  use_model(weights='noisy-student',
                                 include_top=False, pooling='avg',
                                 input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(train_labels.shape[1], activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=predictions) 
    model.compile(optimizer='nadam', loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    return model

with strategy.scope():    
    model1 = get_model(EfficientNetB7) # тут подставить свою модель
model1.load_weights("/kaggle/input/tf-zoo-models-on-tpu-efficientnetb7/my_ef_net_b7.h5")


In [ ]:
def get_model(use_model):
    base_model =  use_model(weights='imagenet',
                                 include_top=False, pooling='avg',
                                 input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(train_labels.shape[1], activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=predictions) 
    model.compile(optimizer='nadam', loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    return model

with strategy.scope():    
    model2 = get_model(DenseNet201) # тут подставить свою модель
model2.load_weights("/kaggle/input/tf-zoo-models-on-tpu-densenet201/my_dense_net_201.h5")


In [ ]:
def get_model(use_model):
    base_model =  use_model(weights='imagenet',
                                 include_top=False, pooling='avg',
                                 input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(train_labels.shape[1], activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=predictions) 
    model.compile(optimizer='nadam', loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    return model

with strategy.scope():    
    model3 = get_model(InceptionResNetV2) # тут подставить свою модель
model3.load_weights("/kaggle/input/tf-zoo-models-on-tpu/InceptionResNetV2.h5")


# Делаем прогноз тестовых данных и готовим представление данных для проверки

In [ ]:
best_alpha = 0.52
bad_alpha = 0.33
print('Вычисляем предсказания...')
probabilities1 = model1.predict(test_dataset, verbose=1)
#probabilities2 = model2.predict(test_dataset, verbose=1)
probabilities3 = model3.predict(test_dataset, verbose=1)

#probabilities = best_alpha * probabilities1 + (1 - best_alpha - bad_alpha) * probabilities2 + bad_alpha * probabilities3
probabilities = best_alpha * probabilities1 + (1 - best_alpha) * probabilities3 

print(probabilities)

sub.loc[:, 'healthy':] = probabilities
sub.to_csv('submission.csv', index=False)
sub.head()